In [2]:
import pandas as pd
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

### 데이터 전처리

In [3]:
df = pd.read_csv('data/전국_시군구_군집.csv')

In [102]:
df.head()

,시도,시군구,code,average_vector,cluster
0,강원특별자치도,강릉시,42150,[-1.18199806e-02 -3.34065711e-02 -4.17965903e-...,2
1,강원특별자치도,고성군,42820,[-1.28417256e-02 -3.51729308e-02 -4.12012981e-...,1
2,강원특별자치도,동해시,42170,[-1.22481304e-02 -3.31160242e-02 -4.15013863e-...,0
3,강원특별자치도,삼척시,42230,[-1.13522222e-02 -3.37827330e-02 -4.39612169e-...,0
4,강원특별자치도,속초시,42210,[-1.32145558e-02 -3.37821353e-02 -3.82817238e-...,3


In [103]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   시도              250 non-null    object
 1   시군구             250 non-null    object
 2   code            250 non-null    int64 
 3   average_vector  250 non-null    object
 4   cluster         250 non-null    int64 
dtypes: int64(2), object(3)
memory usage: 9.9+ KB


In [104]:
df['area'] = df['시도'] + ' ' + df['시군구']

In [105]:
df = df[['area', 'average_vector', 'cluster']]

In [106]:
df.head()

,area,average_vector,cluster
0,강원특별자치도 강릉시,[-1.18199806e-02 -3.34065711e-02 -4.17965903e-...,2
1,강원특별자치도 고성군,[-1.28417256e-02 -3.51729308e-02 -4.12012981e-...,1
2,강원특별자치도 동해시,[-1.22481304e-02 -3.31160242e-02 -4.15013863e-...,0
3,강원특별자치도 삼척시,[-1.13522222e-02 -3.37827330e-02 -4.39612169e-...,0
4,강원특별자치도 속초시,[-1.32145558e-02 -3.37821353e-02 -3.82817238e-...,3


In [107]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   area            250 non-null    object
 1   average_vector  250 non-null    object
 2   cluster         250 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 6.0+ KB


In [108]:
df.to_csv('data/area_clustering_result.csv', encoding='utf-8-sig', index=False)

### 추천 알고리즘

In [3]:
df = pd.read_csv("data/area_clustering_result.csv")
data = json.load(open('data/survey_data_sample.json'))

In [4]:
df.head()

,area,average_vector,cluster
0,강원특별자치도 강릉시,[-1.18199806e-02 -3.34065711e-02 -4.17965903e-...,2
1,강원특별자치도 고성군,[-1.28417256e-02 -3.51729308e-02 -4.12012981e-...,1
2,강원특별자치도 동해시,[-1.22481304e-02 -3.31160242e-02 -4.15013863e-...,0
3,강원특별자치도 삼척시,[-1.13522222e-02 -3.37827330e-02 -4.39612169e-...,0
4,강원특별자치도 속초시,[-1.32145558e-02 -3.37821353e-02 -3.82817238e-...,3


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   area            250 non-null    object
 1   average_vector  250 non-null    object
 2   cluster         250 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 6.0+ KB


In [6]:
data

{'gender': 'F',
 'age': '6',
 'myarea_1': '전라북도 전주시 덕진구',
 'myarea_2': '충청북도 청주시 상당구',
 'myarea_3': '경상남도 창원시 의창구'}

In [7]:
# 군집화 결과 및 유사도 기반 지역 추천 함수
def recommend_areas(cluster_df, survey_data):
    # 문자열 표현의 average_vector를 실제 numpy 배열로 변환
    cluster_df['average_vector'] = cluster_df['average_vector'].apply(lambda vector_str: np.array([float(num) for num in vector_str[1:-1].split()]))
    similar_areas = pd.DataFrame()
    input_areas = []
    # JSON 파일의 각 지역에 대해
    for myarea in ['myarea_1', 'myarea_2', 'myarea_3']:
        # 지역이 비어있지 않다면
        if survey_data[myarea]:
            # 지역의 군집 값을 cluster에 저장
            cluster = cluster_df[cluster_df['area'] == survey_data[myarea]]['cluster'].values[0]
            # 지역의 평균 벡터를 area_vector에 저장
            area_vector = cluster_df[cluster_df['area'] == survey_data[myarea]]['average_vector'].values[0]
            # 지역과 같은 군집에 속하는 모든 지역의 행을 same_cluster에 저장
            same_cluster = cluster_df[cluster_df['cluster'] == cluster].copy()  
            # 지역과 같은 군집의 다른 지역들 사이의 코사인 유사도 계산
            same_cluster['similarity'] = same_cluster['average_vector'].apply(lambda x: cosine_similarity(area_vector.reshape(1, -1), x.reshape(1, -1))[0][0])
            # same_cluster에서 입력 지역 제거
            same_cluster = same_cluster[same_cluster['area'] != survey_data[myarea]]
            # same_cluster를 similar_areas에 추가
            similar_areas = pd.concat([similar_areas, same_cluster])
            # 입력한 지역을 리스트에 추가
            input_areas.append(survey_data[myarea])           
    # 입력한 모든 지역을 similar_areas에서 제거
    for area in input_areas:
        similar_areas = similar_areas[similar_areas['area'] != area]     
    # 중복된 행 제거: 각 지역에 대해 유사도가 가장 높은 행만 남김
    similar_areas = similar_areas.sort_values('similarity', ascending=False).drop_duplicates('area')
    # 입력한 모든 지역과 같은 군집에 속하는 지역들을 유사도가 높은 순으로 정렬하고 상위 5개의 지역 추출
    top_5_areas = similar_areas.sort_values('similarity', ascending=False)['area'].values[:5]
    return top_5_areas

In [8]:
top_5_areas = recommend_areas(df, data)

In [9]:
print(top_5_areas)

['경상남도 창원시 성산구' '전라북도 익산시' '세종특별자치시 세종시' '경기도 수원시 장안구' '경상북도 포항시 남구']
